In [ ]:
import pandas as pd

import sys
import os

sys.path.append(os.path.abspath("../"))

In [45]:
df_kuhp = pd.read_csv("../data/dataset_csv/dataset_kuhp.csv")

In [48]:
df_kuhp["SUB_KLASIFIKASI"] = df_kuhp["SUB_KLASIFIKASI"].map(lambda text: text.replace(" ", "-"))

df_kuhp.head()

,BUKU,BAB,PASAL,ISI_PASAL,SUB_KLASIFIKASI
0,NaN,BAB I,Pasal 1,"(1) Suatu perbuatan tidak dapat dipidana, kec...",tidak-diketahui
1,NaN,BAB I,Pasal 2,Ketentuan pidana dalam perundang-undang an dan...,tidak-diketahui
2,NaN,BAB I,Pasal 3,Ketentuan pidana dalam perundang-undangan Indo...,pelanggaran-lalu-lintas
3,NaN,BAB I,Pasal 4,Ketentuan pidana dalam perundang-undang an Ind...,pemalsuan
4,NaN,BAB I,Pasal 5,(1) Ketentuan pidana dalam perundang-undangan...,tidak-diketahui


In [5]:
df_pidana_umum = pd.read_csv("../data/dataset_csv/dataset_pidana_umum.csv")

In [13]:
import pickle
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

model = tf.keras.models.load_model("../models/model_klasifikasi_pidana.h5", compile=False)

# open file tokenizer.pkl
with open('../models/tokenizer.pkl', 'rb') as file:
   tokenizer = pickle.load(file)

In [16]:
df_pidana_umum = pd.read_csv("../data/dataset_csv/dataset_pidana_umum.csv", encoding="utf-8", usecols=["riwayat_dakwaan", "sub_klasifikasi"])
y = df_pidana_umum["sub_klasifikasi"]

sub_classification = y.unique()
class_mapping = {label: idx for idx, label in enumerate(sub_classification)}
y_encoded = y.map(class_mapping)

def predict_pasal(input_text):
   input_seq = tokenizer.texts_to_sequences([input_text])
   input_padded = pad_sequences(input_seq, maxlen=300, padding='post', truncating='post')
   prediction = model.predict(input_padded)
   predicted_class = sub_classification[prediction.argmax()]
   return predicted_class

In [58]:
predicted_pasal = predict_pasal("terdakwa [Nama Lengkap Terdakwa] dengan sengaja dan tanpa hak mengambil barang milik [Nama Korban], yaitu [Deskripsi Barang yang Dicuri]. Sebelumnya, terdakwa telah merencanakan aksi pencurian dengan membawa peralatan untuk membobol pintu. Setelah berhasil masuk, terdakwa mengambil barang curian dan langsung melarikan diri. Perbuatan terdakwa ini didukung oleh keterangan saksi [Nama Saksi] yang melihat terdakwa keluar dari rumah korban serta barang bukti berupa [Deskripsi Barang Bukti]. Berdasarkan fakta-fakta tersebut, Jaksa Penuntut Umum berpendapat bahwa terdakwa terbukti secara sah dan meyakinkan melakukan tindak pidana pencurian sebagaimana diatur dalam Pasal 362 KUHP dan menuntut pidana penjara selama [Lama Hukuman] tahun, denda sebesar [Jumlah Denda], dan biaya perkara.")

print(predicted_pasal)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
penipuan


In [81]:
pasal = df_kuhp[df_kuhp["SUB_KLASIFIKASI"] == predicted_pasal]

pasal = pasal[["PASAL", "ISI_PASAL"]]

pasal = pasal.to_dict('records')

In [85]:
for p in pasal:
   print(p["PASAL"])
   print(p["ISI_PASAL"])

Pasal 185
Barang siapa dalam perkelahian tanding merampas nyawa pihak lawan atau melukai tubuhnya, maka diterapkan ketentuan-ketentuan mengenai pembunuhan berencana, pembunuhan atau penganiayaan: 1. jika persyaratan tidak diatur terlebih dahulu; 2. jika perkelahian tanding tidak dilakukan di hadapan saksi kedua belah pihak; 3. jika pelaku dengan sengaja dan meru gikan pihak lawan, bersalah melakukan perbuatan penipuan atau yang me nyimpang dari persyaratan.
Pasal 186
(1) Para saksi dan dokter yang menghadiri perkelahian tanding, tidak dipidana. (2) Para saksi diancam: Biro Hukum dan Humas Badan Urusan Administrasi Mahkamah Agung-RI /g3/g3  1. dengan pidana penjara paling lama tiga tahun, jika persyaratan tidak diatur terlebih dahulu, atau jika para saksi menghasut para pihak untuk perkelahian tanding; 2. dengan pidana penjara paling lama empat tahun, jika para saksi dengan sengaja dan merugikan salah satu at au kedua belah pihak, bersalah melakukan perbuatan penipuan atau membiarkan pa